In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [2]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
import pandas as pd
bleu_metric = evaluate.load("sacrebleu")

sari_metric = evaluate.load("sari")


def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_inputs)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

# %%
from torch.utils.data import DataLoader
#tokenizer = MT5Tokenizer.from_pretrained("./mt5")

#load json
with open('../../data/10/test_dissim_dict.json') as f:
    dissim_dict = json.load(f)

test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()

metrics =[]
inpu = []
cands = []
lab = []
for i in tqdm(texts):
        inpu.extend([i.strip()]*len(dissim_dict[i.strip()]))
        cands.extend(dissim_dict[i.strip()])
        lab.extend([[labels[texts.index(i)]]]*len(dissim_dict[i.strip()]))
        #print(len(inpu),len(cands),len(lab))
        eval_arg = [inpu,cands,lab]
        #metric = metrics_func(eval_arg)
        #print(metric)
        #print("++++++++++++++++++++++++++++++")
        #metrics.append(metric)

print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)




/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 10000/10000 [00:00<00:00, 14290.14it/s]


24460 24460 24460
24460 24460 24460
3


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


(13.068031544973188, 32.502832829529666)


In [10]:
tokenizer =  AutoTokenizer.from_pretrained('facebook/bart-base',cahe_dir='/ssd_scratch/cvit/aparna/bart')
model = AutoModelForSeq2SeqLM.from_pretrained("/ssd_scratch/cvit/aparna/mbart_simplification_final")
max_l = 512
num_b = 10
num_sub_b =1
inpu,cands,lab = [],[],[]
f =0
for i in tqdm(texts):
        f+=1
        print(i)
        #inpu.extend([i.strip()]*len(dissim_dict[i.strip()]))
        
        for x in dissim_dict[i.strip()]:
            print(x)
            text = x.strip()
            inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

            # generate text until the output length (which includes the context length) reaches 50
            beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=1,
                top_k=4, top_p=0.95
                # return_dict_in_generate=True,
            )
            cands.extend([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in beam_outputs])
   # print("====================================")
            inpu.extend([text])
            lab.extend([[labels[texts.index(i)]]])
        #print(len(inpu),len(cands),len(lab))
        eval_arg = [inpu,cands,lab]
        #metric = metrics_func(eval_arg)
        #print(metric)
        #print("++++++++++++++++++++++++++++++")
        #metrics.append(metric)
        if f%10 ==0:
            print(i)
            print(len(inpu),len(cands),len(lab))
            metric = metrics_func(eval_arg)
            print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)

  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


However , this option is susceptible to certain chosen-plaintext or known-plaintext attacks , and thus it is designated by NIST to have only 80 bits of security .
This option is susceptible to certain chosen-plaintext or known-plaintext attacks .
However it is designated by NIST .


  0%|          | 1/10000 [00:01<4:58:38,  1.79s/it]

They were the founders and the ruling dynasty of the kingdom of Macedon from about 700 to 310 BC .
They were the founders .
They were the ruling dynasty of the kingdom of Macedon from about 700 to 310 BC .


  0%|          | 2/10000 [00:03<4:33:11,  1.64s/it]

...

 15%|█▍        | 1451/10000 [1:48:19<11:13:11,  4.72s/it]

When a game concludes , the winner is determined by counting each player 's surrounded territory along with captured stones and komi ( points added to the score of the player with the white stones as compensation for playing second ) .
The winner is determined by counting each player 's surrounded territory along with captured stones and komi -LRB- points added to the score of the player with the white stones as compensation for playing second -RRB- .
A game concludes .


 15%|█▍        | 1452/10000 [1:48:22<10:23:02,  4.37s/it]

Sindh is a fertile region and often called the " breadbasket " of the country .
Sindh is a fertile region .
Sindh often called the '' breadbasket '' of the country .


 15%|█▍        | 1453/10000 [1:48:25<8:57:27,  3.77s/it] 

In [2]:
print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)

24460 24460 24460
24460 24460 24460
3


That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


(13.068031544973188, 32.502832829529666)


In [5]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-small-finetuned-text-simplification",cache_dir ="/scratch/aparna")
max_l = 512
num_b = 10
num_sub_b =1
inpu,cands,lab = [],[],[]
f =0
for i in tqdm(texts):
        f+=1
        print(i)
        #inpu.extend([i.strip()]*len(dissim_dict[i.strip()]))
        
        for x in dissim_dict[i.strip()]:
            print(x)
            text = x.strip()
            inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

            # generate text until the output length (which includes the context length) reaches 50
            beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=1,
                top_k=4, top_p=0.95
                # return_dict_in_generate=True,
            )
            cands.extend([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in beam_outputs])
   # print("====================================")
            inpu.extend([text])
            lab.extend([[labels[texts.index(i)]]])
        #print(len(inpu),len(cands),len(lab))
        eval_arg = [inpu,cands,lab]
        #metric = metrics_func(eval_arg)
        #print(metric)
        #print("++++++++++++++++++++++++++++++")
        #metrics.append(metric)
        if f%10 ==0:
            print(i)
            print(len(inpu),len(cands),len(lab))
            metric = metrics_func(eval_arg)
            print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)

  0%|          | 0/10000 [00:00<?, ?it/s]/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


However , this option is susceptible to certain chosen-plaintext or known-plaintext attacks , and thus it is designated by NIST to have only 80 bits of security .
This option is susceptible to certain chosen-plaintext or known-plaintext attacks .
However it is designated by NIST .


  0%|          | 1/10000 [00:01<2:54:01,  1.04s/it]

They were the founders and the ruling dynasty of the kingdom of Macedon from about 700 to 310 BC .
They were the founders .
They were the ruling dynasty of the kingdom of Macedon from about 700 to 310 BC .


  0%|          | 2/10000 [00:01<2:32:01,  1.10it/s]

...

It was completed in 2006 .
It was completed in 2006 .


 19%|█▉        | 1898/10000 [57:12<4:12:15,  1.87s/it]

The area went through several administrative changes before becoming organized as a territory on May 11 , 1912 .
The area went through several administrative changes .
The area was becoming organized as a territory on May 11 , 1912 .


 19%|█▉        | 1899/10000 [57:14<4:22:55,  1.95s/it]

Foster was born in Victoria , British Columbia , the son of Maurice ( Morry ) Foster , a maintenance yard superintendent , and Eleanor May ( Vantreight ) Foster , a homemaker .
Foster was born in Victoria the son of Maurice -LRB- Morry -RRB- Foster .
Foster was born in Victoria the son of a maintenance yard superintendent .
Foster was born in Victoria the son of Eleanor May -LRB- Vantreight -RRB- Foster .
Victoria the son of Maurice -LRB- Morry -RRB- Foster were a homemaker .
A maintenance yard superintendent were a homemaker .
Eleanor May -LRB- Vantreight -RRB- Foster were a homemaker .
Victoria was British Columbia .
